In [1]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [17]:
housing = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col = 0)
real_estate = pd.read_csv('data/Ames_Real_Estate_Data.csv')
housing_df = pd.merge(housing,real_estate,left_on='PID',right_on='MapRefNo',how='inner')

house_df = pd.read_csv('data/final_df.csv', index_col = 0)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# Checking house price dataset, contains 2580 observations
housing.shape

(2580, 91)

In [19]:
# Checking final_df after generating geoCode, found duplicated observations
house_df.shape

(2603, 179)

In [20]:
# Test cell, ignored! 
house_df_test = house_df.copy()
len(house_df_test.drop_duplicates(subset=['PID'],keep = 'first'))

2558

In [21]:
# drop duplicated observations
house_df = house_df.drop_duplicates(subset=['PID'],keep = 'first')
house_df.shape

(2558, 179)

In [22]:
len(house_df[house_df['YrSold']==2010])

314

## Yi's Feature engineer

In [23]:
# calculate age of building
house_df['BldgAge'] = house_df['YrSold'] - house_df['YearBuilt']

# convert MSSubClass to str
house_df['MSSubClass'] = house_df[['MSSubClass']].astype('str')
house_df['TxD_S'] = house_df[['TxD_S']].astype('str')
house_df['SchD_S'] = house_df[['SchD_S']].astype('str')

# convert NA's to 0s in LotFrontage
house_df['LotFrontage'] = np.where(np.isnan(house_df['LotFrontage']), 0, house_df['LotFrontage'])

# binarize YearRemodAdd
house_df['Remodeled'] = np.where(house_df['YearRemodAdd'] == house_df['YearBuilt'], 0, 1)

# binarize Alley
house_df['Alley'] = np.where(pd.isnull(house_df['Alley']), 0, 1)

# binarize MSSubClass to PUD or not PUD
house_df['IsPUD'] = np.where(house_df['MSSubClass'].isin(['120','150','160','180']), 1, 0)

# binarize LotShape to Reg or not Reg
house_df['LotIsReg'] = np.where(house_df['LotShape']=='Reg', 1, 0)

# binarize Condition1/2 to positive feature or no positive feature
house_df['PosFeat'] = np.where(house_df['Condition1'].isin(['PosN','PosA'])|\
                                house_df['Condition2'].isin(['PosN','PosA']), 1, 0)

# binarize condition 1/2 to normal sorroundings or not normal surroundings
house_df['SurrIsNormal'] = np.where((house_df['Condition1']=='Norm') & (house_df['Condition2']=='Norm'), 1, 0)

# binarize condition 1/2 to normal sale condition or not sale condition
house_df['Salecondition'] = np.where((house_df['SaleCondition']=='Normal'), 1, 0)

house_df['ExtMatl'] = np.where((house_df['Exterior1st']==house_df['Exterior2nd']),house_df['Exterior1st'], 'Mixed')


## Daniel's Feature engineer

In [24]:
house_df['BsmtExposure2']=np.where((house_df['BsmtExposure']=='Av')|(house_df['BsmtExposure']=='Gd'),1,0)



## Danny's Feature engineer

In [25]:
house_df['Total_porch_sf'] = house_df['OpenPorchSF'] + house_df['3SsnPorch'] + house_df['EnclosedPorch'] +\
                            house_df['ScreenPorch'] + house_df['WoodDeckSF']

### These are the features I selected from the first 27 features: 
- GrLivArea
- LotArea
- BldgAge
- Remodeled
- IsPUD
- LotIsReg
- PosFeat
- SurrIsNormal (unsure about this one)

In [26]:
house_df.corr()

,index,PID,GrLivArea,SalePrice,Distance,LotFrontage,LotArea,Alley,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MasVnrArea2,GarageArea2,PoolArea2,HalfBath2,BsmtHalfBath2,BasmtFinSF1,BasmtFinSF2,BasmtFinSF,total_LivArea,num_bathroom,GeoRefNo,Tier,Range,MA_Line2,MA_Zip1,MA_Zip2,Rcrd_Yr,Rcrd_Mo,Inst1_Yr,Inst1_Mo,LndAc_S,ImpAc_S,OthAc_S,TtlVal_AsrYr,X2TPr_D,X2TSc_D,X2TPr_S,X2TSc_S,LndAcX1S,ImpAcX1S,ImpAcX2S,HSTtl_D,MilVal_D,HSTtl_S,MilVal_S,AcreX_S1,AcreGr,AcreNt_S,BldgNo_S,DwlgNo_S,YrBuilt,GLA,TtlBsmtSF,GarYrBlt,Cars,YrSold_YYYY,MoSold_MM,PA-Nmbr,PA-PostD,NmbrBRs,address3,latitude2,longitude2,altitude2,ISU_lat,ISU_long,BldgAge,Remodeled,IsPUD,LotIsReg,PosFeat,SurrIsNormal,Salecondition,BsmtExposure2,Total_porch_sf
index,1.000000e+00,2.791739e-02,-1.261419e-02,-5.739388e-02,-2.427344e-02,-2.894691e-02,-3.329784e-02,7.034381e-04,-4.428859e-02,3.092424e-02,-5.361013e-02,-4.334595e-02,-3.161820e-02,-4.813020e-02,2.550290e-02,-5.648592e-03,-4.471576e-02,-3.418459e-02,1.391125e-02,1.790329e-02,-1.857289e-02,1.160716e-02,-5.058203e-03,-1.928123e-02,3.279934e-02,1.963806e-02,1.298555e-02,-1.102082e-02,-4.594916e-02,-4.044620e-02,-4.090368e-02,3.144988e-03,-2.671177e-02,-1.581100e-02,-2.880524e-02,1.735681e-02,7.539172e-03,-5.091314e-03,-2.684664e-02,3.689590e-02,-1.606140e-02,-3.984139e-02,7.539172e-03,-1.659909e-02,1.862167e-02,-5.715182e-02,1.034314e-02,-5.359411e-02,-4.152677e-02,-1.946676e-02,2.791739e-02,NaN,NaN,NaN,-2.712432e-02,2.387474e-02,2.733173e-02,-1.727746e-02,2.733173e-02,-1.727746e-02,-6.837646e-02,NaN,-6.538777e-02,-6.619668e-02,NaN,NaN,NaN,NaN,-1.626887e-02,1.419935e-02,NaN,-4.009877e-03,NaN,-3.215938e-03,NaN,-2.663358e-02,-2.993367e-03,-2.496089e-02,NaN,2.637898e-02,4.377950e-03,-1.233659e-02,-4.134013e-02,-1.520349e-02,-4.402276e-02,1.343176e-02,-2.036210e-02,1.443090e-02,NaN,3.544494e-02,1.443090e-02,-2.112852e-02,1.709440e-02,NaN,4.625062e-14,1.895199e-14,5.523547e-02,1.922232e-03,-1.925056e-02,1.681331e-02,1.572690e-03,-2.253894e-02,2.841051e-01,-5.301140e-02,-1.294672e-02
PID,2.791739e-02,1.000000e+00,-1.035972e-01,-2.280929e-01,-1.759347e-01,-3.066857e-02,4.140111e-02,9.096190e-02,-2.475504e-01,1.218971e-01,-3.379339e-01,-1.285462e-01,-2.287169e-01,-1.120034e-01,-1.270685e-03,-7.265161e-02,-1.901503e-01,-1.543005e-01,1.133257e-02,5.905988e-02,-4.080844e-02,2.159106e-04,-1.559727e-01,-1.749148e-01,2.151015e-03,4.928582e-02,-7.507457e-02,-1.065517e-01,-2.580216e-01,-2.207168e-01,-1.984560e-01,-3.987737e-02,-5.932387e-02,1.715473e-01,-2.064363e-02,-3.075141e-02,-6.196802e-03,-2.198603e-02,-6.365961e-02,4.873327e-03,-2.338928e-01,-1.991345e-01,-6.196802e-03,-1.730355e-01,-2.187923e-04,-9.993216e-02,2.911749e-02,-9.067746e-02,-1.250181e-01,-1.891450e-01,1.000000e+00,NaN,NaN,NaN,1.272370e-02,5.082690e-01,6.735089e-03,-1.124304e-02,6.735089e-03,-1.124304e-02,-2.188482e-01,NaN,-2.174008e-01,-2.179672e-01,NaN,NaN,NaN,NaN,4.016809e-02,1.839779e-02,NaN,-3.889049e-02,NaN,-2.245859e-02,NaN,3.573399e-02,-3.972935e-03,3.621383e-02,NaN,-4.047514e-02,-1.833764e-01,-9.640373e-02,-1.862921e-01,-1.537132e-01,-2.149999e-01,5.513727e-02,-5.369770e-03,-1.692794e-01,NaN,-1.571142e-03,-1.692794e-01,-8.054421e-01,-2.050632e-01,NaN,6.760472e-16,1.131389e-15,3.380931e-01,2.227591e-01,-1.462217e-01,1.300698e-01,-7.540392e-02,4.872405e-02,6.993464e-02,3.878275e-02,-1.829076e-03
GrLivArea,-1.261419e-02,-1.035972e-01,1.000000e+00,7.198022e-01,1.797821e-02,1.467564e-01,2.611281e-01,-2.800647e-03,5.600393e-01,-1.109892e-01,2.287499e-01,3.037251e-01,3.886588e-01,1.642282e-01,-2.042461e-02,2.438666e-01,4.095695e-01,5.389424e-01,6.644188e-01,8.921815e-02,4.247845e-02,-5.339892e-02,6.435057e-01,4.446041e-01,5.371289e-01,1.301704e-01,8.066151e-01,4.570

In [27]:
pd.options.display.max_rows = None
house_df.corr()['SalePrice'].sort_values()

BldgAge          -5.442518e-01
LotIsReg         -3.016197e-01
longitude2       -2.547675e-01
MA_Zip2          -2.321861e-01
PID              -2.280929e-01
GeoRefNo         -2.280929e-01
Salecondition    -1.306464e-01
EnclosedPorch    -1.202169e-01
KitchenAbvGr     -1.152058e-01
Rcrd_Yr          -1.144998e-01
Inst1_Yr         -1.144998e-01
Alley            -1.135083e-01
OverallCond      -1.066500e-01
YrSold_YYYY      -6.320549e-02
index            -5.739388e-02
Remodeled        -5.251723e-02
BsmtHalfBath2    -3.622162e-02
Inst1_Mo         -3.603198e-02
Rcrd_Mo          -3.603198e-02
BsmtHalfBath     -3.400554e-02
IsPUD            -3.187890e-02
LowQualFinSF     -3.126661e-02
MiscVal          -1.923813e-02
ImpAcX1S         -9.399524e-03
MA_Zip1          -7.395858e-03
ISU_long          4.820486e-16
ISU_lat           6.908338e-16
YrSold            9.976053e-04
MoSold            1.382741e-02
AcreGr            1.480496e-02
HSTtl_S           1.771794e-02
MoSold_MM         1.832749e-02
BsmtFinS

In [28]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2580 entries, 1 to 763
Data columns (total 91 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PID            2580 non-null   int64  
 1   GrLivArea      2580 non-null   int64  
 2   SalePrice      2580 non-null   int64  
 3   MSSubClass     2580 non-null   int64  
 4   MSZoning       2580 non-null   object 
 5   LotFrontage    2118 non-null   float64
 6   LotArea        2580 non-null   int64  
 7   Street         2580 non-null   object 
 8   Alley          168 non-null    object 
 9   LotShape       2580 non-null   object 
 10  LandContour    2580 non-null   object 
 11  Utilities      2580 non-null   object 
 12  LotConfig      2580 non-null   object 
 13  LandSlope      2580 non-null   object 
 14  Neighborhood   2580 non-null   object 
 15  Condition1     2580 non-null   object 
 16  Condition2     2580 non-null   object 
 17  BldgType       2580 non-null   object 
 18  HouseStyl

In [29]:
real_estate.isnull().sum()

MapRefNo            0
GeoRefNo            0
Tier                0
Range               0
Prop_Addr          20
ZngCdPr           106
ZngCdSc         22187
ZngOLPr         20243
ZngOLSc         22060
ClassPr_S           0
ClassSc_S           0
Legal_Pr            0
SchD_S              0
TxD_S               0
MA_Ownr1            0
MA_Ownr2        19337
MA_Line1           14
MA_Line2        22213
MA_City            14
MA_State           14
MA_Zip1            14
MA_Zip2          8890
Rcrd_Yr          1907
Rcrd_Mo          1907
Inst1_No         1857
Inst1_Yr         1907
Inst1_Mo         1907
Inst1TPr         2677
LndAc_S             0
ImpAc_S             0
OthAc_S             0
TtlVal_AsrYr        0
ValType             0
X1TPr_D         21519
X1TSc_D         21519
X2TPr_D         22212
X2TSc_D         22212
X1TPr_S         21522
X1TSc_S         21522
X2TPr_S         22212
X2TSc_S         22212
LndAcX1S            0
ImpAcX1S            0
ImpAcX2S            0
HSTtl_D         13024
MilVal_D  

In [31]:
pd.set_option('display.max_rows',None)

X_train.isnull().sum()

NameError: name 'X_train' is not defined

## Features that drop because of too many missing values
- PoolArea
- PoolQC
- MiscFeature
- MiscVal
- YrSold_YYYY
- MoSold_MM
- SaleCond
- PA-PreD
- PA-PostD
- PA-UnTyp
- PA-UntNo
- X1TPr_D
- X1TSc_D
- X2TPr_D
- X2TSc_D
- X1TPr_S
- X1TSc_S
- X2TPr_S
- X2TSc_S

## Features that drop because of multicollinearity
- ISU_lat_long
- address_lat_long
- FullBath 
- HalfBath
- MA_Ownr1
- MA_Ownr2
- MA_Line1
- MA_Line2
- MA_City
- MA_State
- location
- point


## Features that drop because of p value
- MasVnrArea2
- BsmtQual
- BsmtCond
- BsmtExposure
- Electrical

In [36]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [53]:
house_df_test = house_df.drop(['Alley','PoolArea','PoolQC','MiscFeature','MiscVal','YrSold_YYYY','MoSold_MM','SaleCond','PA-PreD','PA-PostD',\
                        'PA-UnTyp','PA-UntNo','X1TPr_D','X1TSc_D','X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S','X2TPr_S','X2TSc_S',\
                       'ISU_lat_long','address','FullBath','HalfBath','MA_Ownr1','MA_Ownr2','MA_Line1','MA_Line2',\
                        'MA_City','MA_State','address3','location2','point2','Street','index','PID','Utilities','BsmtHalfBath2',\
                        'HalfBath2','BsmtFullBath','BsmtHalfBath','BsmtFinSF1','BsmtFinSF2','BsmtFinType1','BsmtFinType2',\
                        'Source','Date','ParType','BldgNo_S','DwlgNo_S','YrBuilt','Ext1','Ext2','Central Air','GLA','TtlBsmtSF',\
                        'GarYrBlt','Cars','MA_Zip1','MA_Zip2','ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','PA-Nmbr','PA-Strt',\
                        'PA-StSfx','Inst1_No','Inst1_Yr','Inst1_Mo','Inst1TPr','TtlVal_AsrYr','ValType','OthAc_S','ImpAc_S',\
                        'LndAc_S','Prop_Addr','HSTtl_D','MilVal_D','HSTtl_S','MilVal_S','GeoRefNo','Tier','Range','AcreX_S1',\
                        'ClassPr_S','ClassSc_S','LndAcX1S','ImpAcX1S','ImpAcX2S','AcreGr','AcreNt_S','ParclRel','WoodDeckSF',\
                        'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','SaleCondition','Rcrd_Yr','Rcrd_Mo','Legal_Pr',\
                        'GrLivArea','Condition1','Condition2','1stFlrSF','2ndFlrSF','YearRemodAdd','YearBuilt','MasVnrType',\
                        'MasVnrArea','GarageArea2','PoolArea2','BsmtQual','BsmtCond','BsmtUnfSF','LowQualFinSF','BasmtFinSF2',\
                        'BasmtFinSF1','NmbrBRs','altitude2','LotShape','Neighborhood','HouseStyle','BedroomAbvGr','num_bathroom',\
                        'MSSubClass','BsmtExposure','TotalBsmtSF','GarageYrBlt','Exterior1st','Exterior2nd','address2',\
                        'MSZoning','latitude2','longitude2','SchD_S','KitchenQual','ExterCond','ISU_lat','ISU_long',\
                        'Functional','TotRmsAbvGrd',],axis=1)
house_df_test.info()
house_df_test.corr()['SalePrice'].sort_values()
# house_df_test.isnull().sum()

# Impute None to Nan value in columns
impute_feature1 = house_df_test[['FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','Fence']]
for feature in impute_feature1.columns:
    house_df_test[feature] = house_df_test[feature].fillna('None')
    
house_df_test = house_df_test.drop(2465,axis=0)
house_df_test = house_df_test.drop(437,axis=0)

house_df_test.isnull().sum()

# Train-test split 
train = house_df_test[house_df_test['YrSold']!=2010]
test = house_df_test[house_df_test['YrSold']==2010]
print(train.shape)
print(test.shape)
X_train = train.drop(['SalePrice'],axis=1)
y_train = train['SalePrice']
X_test = test.drop(['SalePrice'],axis=1)
y_test = test['SalePrice']

X_train_dum = pd.get_dummies(X_train, drop_first=True)
print(X_train.shape)
print(len(X_train_dum.columns))

# apply log to area and house price
X_train_dum['LotFrontage'] = np.log(X_train_dum['LotFrontage']+1)
# X_train_dum['LotArea'] = np.log(X_train_dum['LotArea']+1)
X_train_dum['total_LivArea'] = np.log(X_train_dum['total_LivArea']+1)
X_train_dum['Total_porch_sf'] = np.log(X_train_dum['Total_porch_sf']+1)
y_train = np.log(y_train)

lin_reg = LinearRegression().fit(X_train_dum, y_train)
lin_reg.score(X_train_dum,y_train)



x_feature = sm.add_constant(X_train_dum)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

X_vif = X_train_dum[pValue[pValue<0.05].drop('const').index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2558 entries, 0 to 2602
Data columns (total 46 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SalePrice       2558 non-null   int64  
 1   Distance        2558 non-null   float64
 2   LotFrontage     2558 non-null   float64
 3   LotArea         2558 non-null   int64  
 4   LandContour     2558 non-null   object 
 5   LotConfig       2558 non-null   object 
 6   LandSlope       2558 non-null   object 
 7   BldgType        2558 non-null   object 
 8   OverallQual     2558 non-null   int64  
 9   OverallCond     2558 non-null   int64  
 10  RoofStyle       2558 non-null   object 
 11  RoofMatl        2558 non-null   object 
 12  ExterQual       2558 non-null   object 
 13  Foundation      2558 non-null   object 
 14  Heating         2558 non-null   object 
 15  HeatingQC       2558 non-null   object 
 16  CentralAir      2558 non-null   object 
 17  Electrical      2557 non-null   o

BldgAge          -0.544252
LotIsReg         -0.301620
Salecondition    -0.130646
KitchenAbvGr     -0.115206
OverallCond      -0.106650
Remodeled        -0.052517
IsPUD            -0.031879
YrSold            0.000998
MoSold            0.013827
SurrIsNormal      0.096833
PosFeat           0.125170
LotFrontage       0.183032
Distance          0.191794
LotArea           0.277509
BsmtExposure2     0.354009
MasVnrArea2       0.386136
Total_porch_sf    0.393583
Fireplaces        0.487162
BasmtFinSF        0.551436
GarageArea        0.636363
GarageCars        0.639794
OverallQual       0.790448
total_LivArea     0.820051
SalePrice         1.000000
Name: SalePrice, dtype: float64

SalePrice         0
Distance          0
LotFrontage       0
LotArea           0
LandContour       0
LotConfig         0
LandSlope         0
BldgType          0
OverallQual       0
OverallCond       0
RoofStyle         0
RoofMatl          0
ExterQual         0
Foundation        0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        0
KitchenAbvGr      0
Fireplaces        0
FireplaceQu       0
GarageType        0
GarageFinish      0
GarageCars        0
GarageArea        0
GarageQual        0
GarageCond        0
PavedDrive        0
Fence             0
MoSold            0
YrSold            0
SaleType          0
MasVnrArea2       0
BasmtFinSF        0
total_LivArea     0
TxD_S             0
BldgAge           0
Remodeled         0
IsPUD             0
LotIsReg          0
PosFeat           0
SurrIsNormal      0
Salecondition     0
ExtMatl           0
BsmtExposure2     0
Total_porch_sf    0
dtype: int64

(2242, 46)
(314, 46)
(2242, 45)
120


0.9184228757131621

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     202.6
Date:                Sat, 28 Nov 2020   Prob (F-statistic):               0.00
Time:                        15:55:56   Log-Likelihood:                 1784.0
No. Observations:                2242   AIC:                            -3330.
Df Residuals:                    2123   BIC:                            -2650.
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 11.3166      4

const                 1.151790e-02
Distance              6.957098e-15
LotArea               1.418851e-16
OverallQual           2.879209e-78
OverallCond           3.495586e-48
Fireplaces            9.686927e-09
GarageCars            1.414246e-02
GarageArea            1.045198e-10
BasmtFinSF            4.910214e-32
total_LivArea        6.218876e-204
BldgAge               4.382104e-30
LotIsReg              1.389483e-02
PosFeat               9.529080e-07
SurrIsNormal          2.516685e-13
Salecondition         1.444250e-04
BsmtExposure2         7.311831e-05
Total_porch_sf        2.697526e-03
LandContour_HLS       1.299628e-04
LandContour_Lvl       2.227785e-02
LandSlope_Mod         3.953542e-03
LandSlope_Sev         2.945125e-03
BldgType_2fmCon       1.329780e-03
BldgType_Duplex       5.032628e-03
BldgType_Twnhs        1.323527e-03
RoofMatl_WdShngl      7.569903e-03
ExterQual_Fa          8.232561e-05
ExterQual_Gd          3.718748e-06
ExterQual_TA          1.497897e-08
Foundation_PConc    

              feature         VIF
0            Distance   16.038015
1             LotArea    4.263066
2         OverallQual   70.927145
3         OverallCond   38.492015
4          Fireplaces   10.773061
5          GarageCars   38.879297
6          GarageArea   32.451859
7          BasmtFinSF    3.131391
8       total_LivArea  393.548578
9             BldgAge   11.994065
10           LotIsReg    3.343739
11            PosFeat    1.278835
12       SurrIsNormal    9.348573
13      Salecondition   29.936926
14      BsmtExposure2    1.837632
15     Total_porch_sf    7.101155
16    LandContour_HLS    1.776506
17    LandContour_Lvl   22.431807
18      LandSlope_Mod    1.400092
19      LandSlope_Sev    1.513611
20    BldgType_2fmCon    1.186182
21    BldgType_Duplex    1.293425
22     BldgType_Twnhs    1.194879
23   RoofMatl_WdShngl    1.058009
24       ExterQual_Fa    1.847927
25       ExterQual_Gd   15.809890
26       ExterQual_TA   36.885223
27   Foundation_PConc    4.515656
28    Foundati

In [ ]:
house_df_test.isnull().sum()

In [ ]:
house_df_test.groupby(['Functional']).count()

In [ ]:
# Impute None to Nan value in columns
impute_feature1 = house_df_test[['FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','Fence']]
for feature in impute_feature1.columns:
    house_df_test[feature] = house_df_test[feature].fillna('None')

In [ ]:
house_df_test.isnull().sum()

In [ ]:
house_df_test = house_df_test.drop(2465,axis=0)
house_df_test = house_df_test.drop(437,axis=0)

In [ ]:
house_df_test[house_df_test['Electrical'].isnull()]

In [ ]:
house_df_test[house_df_test['GarageCars'].isnull()]

In [ ]:
house_df_test[house_df_test['TotalBath'].isnull()]

In [ ]:
house_df_test.loc[house_df_test.index == 2328,'TotalBath'] = 3.5

In [ ]:
house_df_test.shape

In [ ]:
house_df_test[house_df_test.index == 918 ]['TotalBath']

In [ ]:
house_df_test.shape

In [ ]:
house_df_test.isnull().sum()

In [ ]:
house_df_test.info()

### Train-test split

In [44]:
# Train-test split 
train = house_df_test[house_df_test['YrSold']!=2010]
test = house_df_test[house_df_test['YrSold']==2010]
print(train.shape)
print(test.shape)
X_train = train.drop(['SalePrice'],axis=1)
y_train = train['SalePrice']
X_test = test.drop(['SalePrice'],axis=1)
y_test = test['SalePrice']

(2242, 44)
(314, 44)


In [45]:
X_train_dum = pd.get_dummies(X_train, drop_first=True)
print(X_train.shape)
print(len(X_train_dum.columns))

(2242, 43)
118


In [46]:
X_train_dum.columns[-50:]

Index(['FireplaceQu_None', 'FireplaceQu_Po', 'FireplaceQu_TA',
       'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn',
       'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_None',
       'GarageFinish_None', 'GarageFinish_RFn', 'GarageFinish_Unf',
       'GarageQual_Fa', 'GarageQual_Gd', 'GarageQual_None', 'GarageQual_Po',
       'GarageQual_TA', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_None',
       'GarageCond_Po', 'GarageCond_TA', 'PavedDrive_P', 'PavedDrive_Y',
       'Fence_GdWo', 'Fence_MnPrv', 'Fence_MnWw', 'Fence_None', 'SaleType_CWD',
       'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_New', 'SaleType_Oth', 'SaleType_VWD', 'SaleType_WD ',
       'TxD_S_45', 'ExtMatl_AsphShn', 'ExtMatl_BrkFace', 'ExtMatl_CBlock',
       'ExtMatl_HdBoard', 'ExtMatl_ImStucc', 'ExtMatl_MetalSd',
       'ExtMatl_Mixed', 'ExtMatl_Plywood', 'ExtMatl_PreCast', 'ExtMatl_Stucco',
       'ExtMatl_VinylSd', 'ExtMatl_Wd Sdng'],
      dt

In [47]:
# apply log to area and house price
X_train_dum['LotFrontage'] = np.log(X_train_dum['LotFrontage']+1)
# X_train_dum['LotArea'] = np.log(X_train_dum['LotArea']+1)
X_train_dum['total_LivArea'] = np.log(X_train_dum['total_LivArea']+1)
X_train_dum['Total_porch_sf'] = np.log(X_train_dum['Total_porch_sf']+1)
y_train = np.log(y_train)

lin_reg = LinearRegression().fit(X_train_dum, y_train)
lin_reg.score(X_train_dum,y_train)

0.9014220574129522

In [48]:
house_df_test_dum = pd.get_dummies(house_df_test, drop_first=True)

house_df_test_dum['LotFrontage'] = np.log(house_df_test_dum['LotFrontage']+1)
# house_df_test_dum['LotArea'] = np.log(house_df_test_dum['LotArea']+1)
house_df_test_dum['total_LivArea'] = np.log(house_df_test_dum['total_LivArea']+1)
house_df_test_dum['Total_porch_sf'] = np.log(house_df_test_dum['Total_porch_sf']+1)
house_df_test_dum['SalePrice'] = np.log(house_df_test_dum['SalePrice'])

train1 = house_df_test_dum[house_df_test['YrSold']!=2010]
test1 = house_df_test_dum[house_df_test['YrSold']==2010]
# print(train1.shape)
# print(test1.shape)
X_train1 = train1.drop(['SalePrice'],axis=1)
y_train1 = train1['SalePrice']
X_test1 = test1.drop(['SalePrice'],axis=1)
y_test1 = test1['SalePrice']


# linear regresion model
lin_reg1 = LinearRegression().fit(X_train1, y_train1)
print(f'R^2 of Train set: {lin_reg1.score(X_train1,y_train1)}')
print(f'R^2 of Test set: {lin_reg1.score(X_test1,y_test1)}')

R^2 of Train set: 0.9014220574129522
R^2 of Test set: 0.828677627011333


In [49]:
# Decision Tree Model
tree_reg = DecisionTreeRegressor(max_depth=10).fit(X_train1,y_train1)
print(f'R^2 of Train set: {tree_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {tree_reg.score(X_test1,y_test1)}')



R^2 of Train set: 0.9644156075606799
R^2 Test set: 0.7224647639522689


In [50]:
# Random Forest Model
forest_reg = RandomForestRegressor(n_estimators=100,max_features=5).fit(X_train1,y_train1)
print(f'R^2 of Train set: {forest_reg.score(X_train1,y_train1)}')
print(f'R^2 Test set: {forest_reg.score(X_test1,y_test1)}')

R^2 of Train set: 0.9769357687919005
R^2 Test set: 0.7631898385412298


In [51]:
import statsmodels.api as sm
x_feature = sm.add_constant(X_train_dum)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     167.5
Date:                Sat, 28 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:14:20   Log-Likelihood:                 1571.8
No. Observations:                2242   AIC:                            -2910.
Df Residuals:                    2125   BIC:                            -2241.
Df Model:                         116                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 10.5685      4

const                  3.170057e-02
Distance               4.148648e-17
LotFrontage            8.971901e-03
OverallCond            1.982452e-54
Fireplaces             3.176940e-12
GarageCars             2.985997e-04
GarageArea             9.559297e-11
MasVnrArea2            7.108003e-03
BasmtFinSF             6.545987e-48
total_LivArea         7.278540e-250
BldgAge                4.051281e-42
LotIsReg               6.968003e-04
PosFeat                8.063277e-07
SurrIsNormal           3.891390e-12
Salecondition          3.154759e-04
BsmtExposure2          2.725090e-07
Total_porch_sf         5.111690e-04
LandContour_HLS        1.235277e-04
BldgType_2fmCon        1.055428e-02
BldgType_Duplex        1.479951e-03
BldgType_Twnhs         6.995538e-04
BldgType_TwnhsE        1.967910e-02
RoofMatl_WdShngl       1.148289e-04
ExterQual_Fa           1.561429e-15
ExterQual_Gd           9.331052e-15
ExterQual_TA           1.787257e-25
Foundation_Slab        4.775095e-07
HeatingQC_Gd           9.793

In [52]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

X_vif = X_train_dum[pValue[pValue<0.05].index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)


KeyError: "['const'] not in index"

In [ ]:
x_feature = sm.add_constant(X_train_dum)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

In [ ]:
X_vif = X_train_dum[pValue[pValue<0.05].index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

In [ ]:
x_feature = sm.add_constant(X_train_dum)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y_train, x_feature)
results_feature = model.fit()
print(results_feature.summary())
pValue = results_feature.pvalues
pValue[pValue<0.05]

In [ ]:
X_vif = X_train_dum[pValue[pValue<0.05].index]
vif_data = pd.DataFrame() 
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                          for i in range(len(X_vif.columns))] 
print(vif_data)

In [ ]:
# from sklearn.metrics import r2_score
# lin_reg_pred = lin_reg.predict(X_test)
# sum((y_test - lin_reg_pred)**2)/sum((y_test - np.mean(y_test))**2)
# mean_squared_error(X_test,y_test)
# lin_reg_pred
# len(y_test)
# r2_score(y_test,lin_reg_pred)

In [ ]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# housing_prediction = lin_reg.predict(X_test)
# mean_squared_error(y_train,y_test)
# lin_reg2 = LinearRegression().fit(X_train,y_train)
# lin_reg.r2_score

### How to make money

In [43]:
import pickle